<a href="https://colab.research.google.com/github/sherjahong1r/Machine-Learning-Lessons/blob/main/03_Pipeline_Sonlar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NEW SECTION**

# **Pipeline. Sonlar**

Mavzu boshida biz jarayonlarni avtomatlashtirish haqida gapirdik. Buning uchun scikit-learn da maxsus **pipeline** tushunchasi bor. Pipeline ingliz tilidan gaz (neft) quvuri deb tarjima qilinadi. Gaz A nuqtadan B nuqtaga yetkazib berilishida bir nechta oraliq ishlov berish stansiyalaridan o'tadi.

Bizning ma'lumotlar ham shunday, boshlang'ich nuqtasidan bevosita MLga yetib kelunga qadar bir nechta jarayonolardan o'tdi. Yuqorida biz har bir jarayonni qo'lda yozib chiqdik, pipeline yordamida esa biz barcha qadamlarni birlashtirib - pipeline (yoki konveyer) hosil qilishimiz mumkin.

Pipeline so'zini konveyer deb tarjima qilishimga sabab, ma'lumotlarimiz huddi konveyerdan o'tgani kabi turli bosqichlarda turli o'zgarishlardan o'tayapti.

Biz konveyerni 2 qismga bo'lamiz:
- Sonli ustunlarga ishlov berish
- Matnli ustunlarga ishlov berish

In [27]:
import pandas as pd
import numpy as np
import sklearn as sk

url = "https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df = pd.read_csv(url)

from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=50)

housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()

housing_num = housing.drop("ocean_proximity", axis=1)


In [28]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [29]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
4450,-118.21,34.07,42.0,902.0,318.0,1312.0,323.0,1.9375,<1H OCEAN
14253,-117.10,32.71,29.0,3422.0,713.0,2775.0,644.0,1.7075,NEAR OCEAN
15338,-117.29,33.20,16.0,2150.0,461.0,1428.0,407.0,2.4754,NEAR OCEAN
18156,-122.06,37.36,35.0,2693.0,493.0,1343.0,455.0,6.0777,<1H OCEAN
13431,-117.42,34.10,18.0,3977.0,809.0,2231.0,742.0,4.1399,INLAND


In [30]:
housing_labels.head()

,median_house_value
4450,168800.0
14253,86900.0
15338,157300.0
18156,327500.0
13431,115400.0


In [31]:
housing_num.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
4450,-118.21,34.07,42.0,902.0,318.0,1312.0,323.0,1.9375
14253,-117.10,32.71,29.0,3422.0,713.0,2775.0,644.0,1.7075
15338,-117.29,33.20,16.0,2150.0,461.0,1428.0,407.0,2.4754
18156,-122.06,37.36,35.0,2693.0,493.0,1343.0,455.0,6.0777
13431,-117.42,34.10,18.0,3977.0,809.0,2231.0,742.0,4.1399


In [32]:
from sklearn.base import BaseEstimator, TransformerMixin

# bizga kerak ustunlar indexlari
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesadder(BaseEstimator, TransformerMixin):
  def __init__(self, add_bedrooms_per_room=True):
    self.add_bedrooms_per_room = add_bedrooms_per_room
  def fit(self, X, y=None):
    return self # bizning funksiyamiz faqat transformer. estimator emas
  def transform(self, X):
    rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
    population_per_household = X[:, population_ix] / X[:, households_ix]
    if self.add_bedrooms_per_room:  # add_bedrooms_per_room ustuni ixtiyoriy bo'ladi
      bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
      return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
    else:
      return np.c_[X, rooms_per_household, population_per_household]

# Ozimiz yaratgan clas bunda yangi ustunlar yaratilgan

sklearn.pipeline.Pipeline — bu ma'lumotlarni qayta ishlash va modelni o'qitish bosqichlarini bitta tartibli zanjirga birlashtiruvchi vosita. U kodni soddalashtiradi, ma'lumotlar 'sızishi'ning oldini oladi va ish jarayonini avtomatlashtiradi.

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('atter_adder', CombinedAttributesadder(add_bedrooms_per_room= True)),
    ('std_scaler', StandardScaler())
])
num_pipeline.fit_transform(housing_num)

# num_pipeline = Pipeline([...]): Bu num_pipeline nomli konveyerni (ketma-ket bajariladigan operatsiyalar zanjirini) yaratadi.
# ('imputer', SimpleImputer(strategy="median")): Bu qadamda, ma'lumotlardagi yetishmayotgan (NaN) qiymatlar 'median'
# (o'rtacha) qiymat bilan to'ldiriladi. Bu yetishmayotgan ma'lumotlarni qayta ishlashning umumiy usuli.
# ('atter_adder', CombinedAttributesadder(add_bedrooms_per_room= True)): Bu qadamda, siz oldinroq yaratgan
# CombinedAttributesadder transformer ishlatiladi. U uy xo'jaligiga to'g'ri keladigan xonalar va aholi soni kabi yangi xususiyatlarni
# qo'shadi, shuningdek, 'add_bedrooms_per_room=True' bo'lganligi sababli, xonalar soniga to'g'ri keladigan yotoqxona soni ham qo'shiladi.
# ('std_scaler', StandardScaler()): Bu qadamda, ma'lumotlar 'StandardScaler' yordamida standartlashtiriladi. Bu xususiyatlarni o'rtacha
# qiymati 0 va standart og'ishi 1 bo'lishi uchun masshtablaydi, bu esa ko'plab mashinani o'rganish algoritmlari uchun foydalidir.
# num_pipeline.fit_transform(housing_num): Bu buyruq num_pipeline konveyerini housing_num ma'lumotlariga qo'llaydi.
# fit metodi transformatorlarni ma'lumotlarga moslashtiradi (masalan, imputer uchun median qiymatini hisoblaydi, scaler uchun o'rtacha
# va standart og'ishni hisoblaydi), transform metodi esa ma'lumotlarga ushbu o'zgarishlarni qo'llaydi va qayta ishlangan ma'lumotlarni qaytaradi.

array([[ 0.67744368, -0.73125093,  1.06514215, ..., -1.03730789,
         0.09095707,  2.11134054],
       [ 1.23079392, -1.36705675,  0.03230036, ..., -0.04596848,
         0.11356918, -0.08204302],
       [ 1.13607631, -1.13797965, -1.00054142, ..., -0.05820121,
         0.04032007,  0.01015281],
       ...,
       [ 0.82201266, -0.79670153, -1.3183389 , ..., -0.10254283,
        -0.11659941, -0.18505824],
       [ 0.64753286, -0.86215213,  0.82679405, ..., -0.45477866,
         0.03643089,  0.4181862 ],
       [ 1.16100199, -0.4881487 , -1.95393385, ...,  0.74609947,
        -0.03143763, -0.83978731]])

Sonli ustunlarga ishlov beruvchi konveyer tayyor, matni ustunlarchi?

Buning uchun maxsus ColumnTransformer obyektiga murojaat qilamiz, bu ham pipeline bir ko'rinishi. ColumnTransformer ichiga biz yuqorida yasalgan num_ipeline ham qo'shib yuboramiz.

In [34]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs)
])

# bu kod ColumnTransformer yordamida ma'lumotlaringizdagi raqamli (num_pipeline bilan) va kategorik
# (OneHotEncoder bilan) ustunlarga alohida-alohida qayta ishlash operatsiyalarini qo'llaydigan to'liq bir jarayonni yaratadi.

In [35]:
housing_prepared = full_pipeline.fit_transform(housing)

# Bu qator full_pipeline (to'liq konveyer) ni housing ma'lumotlariga qo'llaydi. Bunda konveyer ichidagi barcha
# transformatsiyalar (SimpleImputer, CombinedAttributesadder, StandardScaler va OneHotEncoder) housing ma'lumotlariga
# moslashtiriladi (fit) va shu ma'lumotlarga qo'llaniladi (transform). Natijada,
# barcha qayta ishlangan ma'lumotlar housing_prepared o'zgaruvchisiga saqlanadi.

In [36]:
housing_prepared[0:5,:]
# Bu kod housing_prepared o'zgaruvchisining (bu NumPy massivi) dastlabki 5 qatorini (satrini) va barcha ustunlarini ko'rsatadi.
# Bu ma'lumotlarni qayta ishlashdan keyingi holatini tezda ko'rish uchun ishlatiladi.

array([[ 0.67744368, -0.73125093,  1.06514215, -0.80711864, -0.52620638,
        -0.09647019, -0.46622564, -1.01288451, -1.03730789,  0.09095707,
         2.11134054,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.23079392, -1.36705675,  0.03230036,  0.37206038,  0.43031864,
         1.20837307,  0.38915188, -1.13361541, -0.04596848,  0.11356918,
        -0.08204302,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 1.13607631, -1.13797965, -1.00054142, -0.22314427, -0.17992011,
         0.0069897 , -0.24238853, -0.73053165, -0.05820121,  0.04032007,
         0.01015281,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [-1.24183416,  0.80683815,  0.50899657,  0.03094073, -0.10242947,
        -0.06882142, -0.11448161,  1.16037685,  0.19193487, -0.01064924,
        -0.46674974,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.07126952, -0.7172258 , -0

## Ma'lumotlar ML uchun tayyor.